In [ ]:
!pip install transformers datasets==2.14.0 accelerate evaluate pyarrow==15.0.2 fsspec==2023.9.2

INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 3.8 MB/s  0:00:00
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ------------- -------------------------- 3.9/12.0 MB 19.5 MB/s eta 0:00:01
   -------------------------- ------------- 7.9/12.0 MB 19.4 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 20.0 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 17.9 MB/s  0:00:00

  Attempting uninstall: dill

    Found existing installation: dill 0.4.0

    Uninstalling dill-0.4.0:

      Successfully uninstalled dill-0.4.0

   ---------------------

In [1]:
from datasets import load_dataset

# NSMC 데이터셋 로드
# Hugging Face Hub에 호스팅된 nsmc 데이터셋을 불러옵니다.
dataset = load_dataset("nsmc")
print(dataset)

c:\Users\Younghoon\miniconda3\envs\lginnotek\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [2]:
from transformers import AutoTokenizer

# KLUE-BERT 토크나이저 로드
model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    # truncation=True: 모델의 최대 입력 길이(512)를 넘는 문장은 자름
    # padding="max_length": 짧은 문장은 패딩 토큰()으로 채움 (배치 처리를 위해)
    # max_length=128: 영화 리뷰는 대체로 짧으므로 128로 제한하여 학습 속도 향상
    return tokenizer(examples["document"], truncation=True, max_length=128, padding="max_length")

# map 함수를 사용하여 전체 데이터셋에 전처리 적용 (batched=True로 병렬 처리)
encoded_dataset = dataset.map(preprocess_function, batched=True)

c:\Users\Younghoon\miniconda3\envs\lginnotek\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Younghoon\.cache\huggingface\hub\models--klue--bert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 50000/50000 [00:05<00:00, 9245.68 examples/s]


In [4]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# 모델 초기화 (num_labels=2: 긍정/부정)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

# 평가 지표 설정 (정확도)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Logits에서 가장 높은 확률을 가진 클래스 선택 (Argmax)
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# 학습 하이퍼파라미터 설정
args = TrainingArguments(
    output_dir="bert-nsmc-result",
    eval_strategy="epoch",  # 매 에포크마다 평가 수행
    save_strategy="epoch",        # 매 에포크마다 모델 저장
    learning_rate=2e-5,           # 미세 조정 시에는 매우 낮은 학습률 사용 (중요!)
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,           # 보통 3~5 에포크면 수렴함
    weight_decay=0.01,            # AdamW의 가중치 감쇠 적용
    fp16=True,                    # GPU 메모리 절약 및 속도 향상을 위한 16비트 연산 (선택)
)

# Trainer 객체 생성 및 학습 시작
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Younghoon\AppData\Local\Temp\ipykernel_7696\1705750352.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:
trainer.train()

c:\Users\Younghoon\miniconda3\envs\lginnotek\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# 테스트: Fine-tuned 모델로 감성 분석 수행
def predict_sentiment(text, model, tokenizer, device='cuda'):
    """
    텍스트의 감성을 예측하는 함수
    Returns: (label, confidence)
    """
    model.eval()
    
    # 토크나이징
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=128,
        padding='max_length'
    )
    
    # 디바이스로 이동
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        pred = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][pred].item()
    
    label = "긍정 😊" if pred == 1 else "부정 😞"
    return label, confidence

# 테스트할 예시 문장들
test_sentences = [
    "이 영화 정말 재미있어요! 강력 추천합니다.",
    "시간 낭비였다. 다시는 안 볼 듯.",
    "배우들 연기가 훌륭했고 스토리도 감동적이었습니다.",
    "기대했는데 실망이네요. 별로였어요.",
    "그냥 평범한 영화였어요.",
    "인생 영화입니다. 눈물이 멈추지 않았어요.",
    "연출이 너무 지루하고 전개가 느렸다.",
    "OST가 너무 좋았고 영상미도 예술이었어요!"
]

# 모델을 평가 모드로 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print("=" * 60)
print("NSMC Fine-tuned BERT 감성 분석 테스트")
print("=" * 60)

for sentence in test_sentences:
    label, confidence = predict_sentiment(sentence, model, tokenizer, device)
    print(f"\n📝 입력: {sentence}")
    print(f"   → 예측: {label} (확신도: {confidence:.2%})")

print("\n" + "=" * 60)

# 직접 입력 테스트
while True:
    user_input = input("\n테스트할 문장을 입력하세요 (종료: q): ")
    if user_input.lower() == 'q':
        print("테스트 종료!")
        break
    
    label, confidence = predict_sentiment(user_input, model, tokenizer, device)
    print(f"→ 예측: {label} (확신도: {confidence:.2%})")